In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os

PROJ_ROOT = os.pardir

TRAIN_DATA = os.path.join(PROJ_ROOT, "data", "raw", "train.csv")
TEST_DATA = os.path.join(PROJ_ROOT, "data", "raw", "test.csv")

import sys
sys.path.append(os.path.join(PROJ_ROOT, "src"))

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from IPython.display import display


from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
PATH = os.path.join(PROJ_ROOT, "data", "raw/")

PATH

from ml_utilities.ml_utilities import *

from sklearn.metrics import accuracy_score, r2_score

from sklearn.model_selection import train_test_split

In [ ]:
!ls {PATH} 

In [ ]:
torch.cuda.set_device(1)

In [ ]:
sz = 32
arch = resnet18
bs = 50

In [ ]:
label_csv = f'{PATH}train.csv'
n = len(list(open(label_csv))) - 1 # header is not counted (-1)
val_idxs = get_cv_idxs(n) # random 20% data for validation set

In [ ]:
n, len(val_idxs)

In [ ]:
label_df = pd.read_csv(label_csv)

In [ ]:
label_df.head()

In [ ]:
label_df.pivot_table(index="has_cactus", aggfunc=len).sort_values('id', ascending=False)

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}train.csv', test_name='test', # we need to specify where the test set is if you want to submit to Kaggle competitions
                                   val_idxs=val_idxs, tfms=tfms, bs=bs)

In [ ]:
fn = PATH + data.trn_ds.fnames[0]; fn

In [ ]:
img = PIL.Image.open(fn); img

In [ ]:
img.size

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
learn.fit(1e-2, 5, cycle_len=1)

In [ ]:
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds),0)


In [ ]:
y

In [ ]:
probs_y = [np.argmax(x) for x in probs]

In [ ]:
subm = pd.read_csv(f"{PATH}/sample_submission.csv")

In [ ]:
subm.has_cactus = y

In [ ]:
subm.to_csv("../../submission.csv", index=None)